# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import urllib

import json

# Import API key
from api_keys import geoapify_key

import os

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kyshtovka,58.5670,77.1291,3.94,82,0,3.14,RU,1726354047
1,1,gebog,-6.6336,110.8319,20.96,77,70,1.35,ID,1726354048
2,2,vuktyl,63.7153,59.9332,11.52,93,11,1.16,RU,1726354053
3,3,stanley,-51.2581,-57.6930,3.95,84,47,9.06,FK,1726354058
4,4,beroroha,-28.1645,118.9821,11.28,60,0,8.10,AU,1726354065


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', #Longtitute and Latitude as the x and y coordinates
    geo=True, #Enable geographic plotting
    tiles='OSM', #Background map tiles
    size='Humidity', #Point size proportional to humidity
    color='Humidity', #Color scale based on humidity
    frame_width=700, #Map width
    frame_height=500, #Map height
    hover_cols=['City', 'Country', 'Humidity'] #Show additional info on hover
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
#Converted from kelvin to C in WeatherPy.ipynb
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 15) &
    (city_data_df['Max Temp'] <= 30) &
    (city_data_df['Humidity'] <= 75) &
    (city_data_df['Wind Speed'] < 7)
]

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,grytviken,-21.8187,44.9782,21.64,64,0,1.97,MG,1726354066
6,6,invercargill,49.3787,-79.2006,21.61,75,15,2.16,CA,1726354076
9,9,lycksele,40.7780,31.7003,18.09,61,41,1.04,TR,1726354097
21,21,thinadhoo,-9.1729,123.6999,26.24,69,34,0.87,ID,1726354147
24,24,kenai,35.0357,-118.4438,24.69,37,0,5.36,US,1726354154


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Pandas copy function to create DataFrame hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,grytviken,MG,-21.8187,44.9782,64,
6,invercargill,CA,49.3787,-79.2006,75,
9,lycksele,TR,40.7780,31.7003,61,
21,thinadhoo,ID,-9.1729,123.6999,69,
24,kenai,US,35.0357,-118.4438,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel

# Base URL for the Geoapify Places API (for searching places like hotels)
base_url = "https://api.geoapify.com/v2/places"

# Set the search radius (in meters)
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for the current city
    lat = row['Lat']
    lng = row['Lng']

    # Update the params dictionary with the current city coordinates
    #I tried to use filter to find a hotel within radius 10000 meters, but it did not work correctly, so I am searching for nearest hotel.
    params = {
        "categories": "accommodation.hotel",  # Search for hotels
        # "filter": f"circle:{lng},{lat},{radius}",  # Use circle filter with lat/lng and radius
        "bias": f"proximity:{lng},{lat}",  # Proximity bias
        "limit": 1,  # Get only the nearest hotel
        "apiKey": geoapify_key
    }

    full_url = base_url + "?" + urllib.parse.urlencode(params)


    # Make the API request
    response = requests.get(full_url)

    # print(f"{row=} {full_url=} {response.content.decode()}")

    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Convert the API response to JSON format
            name_address = response.json()

            # Extract the hotel name from the response if available
            if name_address['features']:
                hotel_name = name_address['features'][0]['properties']['name']
                # Update the 'Hotel Name' column in the DataFrame
                hotel_df.at[index, 'Hotel Name'] = hotel_name
            else:
                # If no hotel is found, set 'No Hotel Found'
                hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
        except (KeyError, IndexError) as e:
            print(f"Error parsing API response: {e}")
            hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
    else:
        print(f"Error: {response.status_code} for city {row['City']}: {response.content.decode()}")
        hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame
hotel_df.head()


Starting hotel search
grytviken - nearest hotel: Kanto
invercargill - nearest hotel: Motel le Bivouac
lycksele - nearest hotel: Öğretmen Evi
thinadhoo - nearest hotel: Hotel Oé-Upu
kenai - nearest hotel: Baymont Tehachapi
freetown - nearest hotel: Гостиница
bugene - nearest hotel: سوني سفاري
vila velha - nearest hotel: سماء رنية
jiuquan - nearest hotel: Posada del Parque
port douglas - nearest hotel: Knott's Crossing Resort
semporna - nearest hotel: Carousel Holiday Resort
tweed heads - nearest hotel: Tropical hotel
Error parsing API response: 'name'
bushehr - nearest hotel: No Hotel Found
shibin al kawm - nearest hotel: 鱗晃荘
castillos - nearest hotel: Uncle Changs
margate - nearest hotel: The Grove on Russell
gadzhiyevo - nearest hotel: Mango Camp


,City,Country,Lat,Lng,Humidity,Hotel Name
5,grytviken,MG,-21.8187,44.9782,64,Kanto
6,invercargill,CA,49.3787,-79.2006,75,Motel le Bivouac
9,lycksele,TR,40.7780,31.7003,61,Öğretmen Evi
21,thinadhoo,ID,-9.1729,123.6999,69,Hotel Oé-Upu
24,kenai,US,35.0357,-118.4438,37,Baymont Tehachapi


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
#%%capture --no-display
# Create the hover message including City, Country, Hotel Name, and Humidity
hotel_df['Hover Info'] = hotel_df.apply(
    lambda row: f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}<br>Humidity: {row['Humidity']}%", axis=1)

# Plot the points on a map with the hover information
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,  # Activate geographic plot
    color='Humidity',  # Color by Humidity
    size=10,  # Size of the points
    hover_cols=['Hover Info'],  # Show the custom hover message
    tiles='CartoLight',  # Optional: Base map tiles
    frame_width=800,  # Adjust width
    frame_height=600  # Adjust height
)

# Show the map with hover info
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover Info)